# Sunshine List gender analysis

*March 26, 2022*

I had an idea a few months ago to use the sunshine list to look at the gender pay gap. In Canada, there really isn't a whole lot of info on this, but the sunshine list might give us a good opportunity to see what it looks like in the public sector in Canada. The big problem: we don't have genders of the people on the sunshine list. Here's one way we could solve that problem.

First, we'll import pandas and set a global option to display floats with commas (just to make things more readable).

In [68]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format

### Preparing the data

There's a wealth of sunshine list data going back all the way to 2012, so you might wonder why I don't include it all in this analysis. There are two reasons for that:

1. It's hard to compare money over the course of many years, as inflation is a thing.
2. There are likely many duplicate names that are very hard to remove from our dataset if we combine them all.

Therefore it's probably best to just use the latest data available. It's still quite robust!

In [36]:
raw = pd.read_csv("./raw/2021.csv")

Now let's do some cleaning. We'll make our column names all lower case for consistency, and we'll make all the strings in our dataset uppercase. This makes it easier to match the names up on joins later in the analysis.

In [37]:
data = raw.copy()

data.columns = data.columns.str.lower()

for label, content in data[["sector", "last name", "first name", "employer", "job title"]].items():
    data[label] = (data[label]
                        .str.upper()
                        .str.replace("\s(AND){1}\s", " & ", regex=True)
                        .str.replace("\-", "–", regex=True)
                        .str.replace("*", "", regex=False)
                )
    
data.sample(5)

,sector,last name,first name,salary,benefits,employer,job title,year,_docid
158007,SCHOOL BOARDS,DAVIS,GORICA,"103,022.13",0.0,KAWARTHA PINE RIDGE DISTRICT SCHOOL BOARD,ELEMENTARY TEACHER,2021,158007
238723,UNIVERSITIES,ROBERTSON,JEAN,"214,339.02",143.16,UNIVERSITY OF TORONTO,"DIRECTOR OF HUMAN RESOURCES, FACULTY OF MEDICINE",2021,238723
127873,ONTARIO POWER GENERATION,LESSARD,KEITH,"108,517.69","1,367.76",ONTARIO POWER GENERATION,ELECTRICAL & CONTROL TECHNICIAN/TECHNOLOGIST,2021,127873
32400,GOVERNMENT OF ONTARIO – MINISTRIES,ORLANDO,SUSAN,"249,546.4",309.88,ATTORNEY GENERAL,DIRECTOR,2021,32400
119309,MUNICIPALITIES & SERVICES,VAN DER KRABBEN,STEVEN,"130,057.32","1,037.3",CITY OF TORONTO – POLICE SERVICE,POLICE CONSTABLE,2021,119309


Now we're going to read in a dataset of names, downloaded [here](https://data.world/howarder/gender-by-name).

In [38]:
names = pd.read_csv("name_gender.csv")

This dataset has two columns that are important to us: one with a first name, and another with an M or an F, signifying whether it's a male or female name. Now of course this approach is fraught with complications:

1. Some names are common for both men and women. The dataset actually includes a third "probability" column that lists the probability a name is male or female. We don't use it here, but this analysis could be refined by only using names that have a high enough probability.
2. Not everyone identifies as male or female, and their name is a somewhat poor way to identify someone's gender. Given that few other approaches exist, we're trying it this way anyways.

Now we're going to make the data in the name field uppercase so we don't have to worry about cases messing up our matching.

In [39]:

names["name"] = names["name"].str.upper()

We're also going to add another column to our dataset for comparing to the names database. We do this because several names in the sunshine list data have initials following the first name (which would not match to a name that would otherwise match). We also have a line of code here that breaks double names (Mary Jane) into just the first part (Mary) so that we don't have to just throw those names out.

In [40]:
data["first name_cleaned"] = (data["first name"]
                              .str.replace("\s+[A-Z]+\.+", "", regex=True)
                              .str.upper()
                              .dropna()
                              .str.split(" ", 1)
                              .dropna().apply(lambda x: x[0])
                              )

Now we merge the names dataset with the sunshine list data. Note we're coming to our newly cleaned first name column. I'm also calling `.loc` to ensure we only include the columns we want, and in the right order.

In [41]:
data = (data
        .merge(names, left_on='first name_cleaned', right_on="name", how="left")
        .drop_duplicates()
        .loc[:, ["first name", "last name", "gender", "sector", "job title", "employer", "salary", "benefits"]]
        )

We're also going to fill anything that doesn't match with "UNKNOWN".

In [42]:
data["gender"] = data["gender"].fillna("UNKNOWN")

Now let's see what it looks like.

In [43]:
data.sample(5)

,first name,last name,gender,sector,job title,employer,salary,benefits
154667,JASON,CODE,M,SCHOOL BOARDS,SECONDARY TEACHER,DISTRICT SCHOOL BOARD OF NIAGARA,"103,595.85",0.0
114377,CHRISTOPHER,SIMOVIC,M,MUNICIPALITIES & SERVICES,OPERATIONS TECHNICIAN,CITY OF BRAMPTON,"108,991.85",434.34
11327,DIANA,HALLETT,F,CROWN AGENCIES,"DIRECTOR, TRANSPLANT SERVICES/DIRECTRICE, SERV...",ONTARIO HEALTH,"112,567.42",336.48
60097,RICHARD,PARADIS,M,HOSPITALS & BOARDS OF PUBLIC HEALTH,ANESTHESIA ASSISTANT,SINAI HEALTH SYSTEM,"115,413.61",314.4
191721,LINDSAY,MONTADOR,F,SCHOOL BOARDS,TEACHER,PEEL DISTRICT SCHOOL BOARD,"103,200.99",0.0


Now that our dataset is prepared, we can dive into the good stuff.

### Mean salary by sector

Beceause the sunshine list is everyone who makes more than $100K annually, we can find out a lot here by just checking out the counts of men and women on the list. Let's start with that.

In [54]:
gender_counts = data[["gender", "first name"]].groupby("gender").count()

gender_counts

,first name
gender,
F,121163
M,105807
UNKNOWN,17420


There are actually more women than men on our list!

It's also useful to see how many names on our list have been labeled with a gender (versus just labelled Unknown, which is what we relabelled our null values). Of course, we should do some manually spot checking to see how accurate we think the name gendering was, but this will give us a sense of how many values were mapped to something in the dataset.

In [57]:
(gender_counts.loc["F", :] + gender_counts.loc["M", :]) / gender_counts.sum() * 100

first name   92.87204877449977
dtype: float64

Roughly 92% of names in the sunshine list were assigned a gender from the names list! Not bad. Now, back to our analysis. Let's see the counts by sector.

In [50]:
data.pivot_table(index="sector", values="salary", columns="gender", aggfunc="count").sort_values("F", ascending=False)

gender,F,M,UNKNOWN
sector,,,
SCHOOL BOARDS,"53,392.0","22,770.0","4,272.0"
HOSPITALS & BOARDS OF PUBLIC HEALTH,"21,351.0","6,314.0","2,906.0"
MUNICIPALITIES & SERVICES,"14,000.0","36,528.0","3,307.0"
UNIVERSITIES,"9,190.0","10,919.0","2,987.0"
GOVERNMENT OF ONTARIO – MINISTRIES,"7,991.0","10,991.0","1,337.0"
OTHER PUBLIC SECTOR EMPLOYERS,"5,080.0","3,020.0",631.0
CROWN AGENCIES,"4,450.0","4,490.0",884.0
COLLEGES,"3,637.0","3,790.0",437.0
ONTARIO POWER GENERATION,"1,490.0","6,440.0",590.0


There are far more women than men on the list in the school, hospital, and municipalities sectors.

Before we continue, let's remove the seconded sectors here, as they don't seem that interesting and we want to keep our tables readable here.

In [ ]:
data = data[~data["sector"].str.contains("SECONDED")]

Now, we'll take a look at the mean salaries for each sector.

In [69]:
(data
 .pivot_table(index="sector", values="salary", columns="gender", aggfunc="mean")
 .dropna()
 .rename(columns={"salary": "gendered"})
 )

gender,F,M,UNKNOWN
sector,,,
COLLEGES,"119,965.02","119,909.56","116,069.26"
CROWN AGENCIES,"128,613.06","135,230.03","126,394.56"
GOVERNMENT OF ONTARIO – JUDICIARY,"239,394.48","243,610.83","209,683.97"
GOVERNMENT OF ONTARIO – LEGISLATIVE ASSEMBLY & OFFICES,"138,690.75","138,861.28","132,769.40"
GOVERNMENT OF ONTARIO – MINISTRIES,"130,026.95","130,673.86","123,632.89"
HOSPITALS & BOARDS OF PUBLIC HEALTH,"119,190.00","134,527.42","125,276.92"
MUNICIPALITIES & SERVICES,"122,733.62","125,364.40","121,554.09"
ONTARIO POWER GENERATION,"145,029.68","155,543.36","149,466.83"
OTHER PUBLIC SECTOR EMPLOYERS,"129,350.55","140,225.40","131,233.50"


And the median salary too, to get a sense of the average values without the big outliers influencing things.

In [70]:
(data
 .pivot_table(index="sector", values="salary", columns="gender", aggfunc="median")
 .dropna()
 .rename(columns={"salary": "gendered"})
 )

gender,F,M,UNKNOWN
sector,,,
COLLEGES,"115,378.12","115,378.12","115,354.01"
CROWN AGENCIES,"115,918.25","120,310.04","116,419.38"
GOVERNMENT OF ONTARIO – JUDICIARY,"159,266.54","267,013.37","154,173.92"
GOVERNMENT OF ONTARIO – LEGISLATIVE ASSEMBLY & OFFICES,"128,859.79","129,517.58","126,242.05"
GOVERNMENT OF ONTARIO – MINISTRIES,"115,786.98","119,338.34","114,129.41"
HOSPITALS & BOARDS OF PUBLIC HEALTH,"109,873.87","114,076.31","110,821.33"
MUNICIPALITIES & SERVICES,"115,780.48","119,933.02","115,544.70"
ONTARIO POWER GENERATION,"133,665.06","144,675.01","139,950.98"
OTHER PUBLIC SECTOR EMPLOYERS,"117,454.21","122,479.24","115,951.20"


### Mean salary by employer

Now let's break things down by employer instead. The analysis is similar to above.

In [71]:
employers = (data
 .pivot_table(index="employer", values="salary", columns="gender", aggfunc="count")
 .dropna()
 .rename(columns={"salary": "gendered"})
 .sort_values("M", ascending=False)
 )

employers

gender,F,M,UNKNOWN
employer,,,
ONTARIO POWER GENERATION,"1,477.00","6,341.00",583.00
CITY OF TORONTO,"2,059.00","4,716.00",570.00
ONTARIO PROVINCIAL POLICE,"1,331.00","3,924.00",120.00
CITY OF TORONTO – POLICE SERVICE,"1,004.00","3,661.00",360.00
TORONTO DISTRICT SCHOOL BOARD,"6,595.00","3,111.00",843.00
...,...,...,...
CHILDREN’S AID SOCIETY OF THE UNITED COUNTIES OF STORMONT DUNDAS & GLENGARRY,18.00,1.00,2.00
SUDBURY DISTRICT NURSE PRACTITIONER CLINICS,5.00,1.00,1.00
ONTARIO PAROLE BOARD,2.00,1.00,1.00


In [72]:
employers["total_gendered"] = employers["M"] + employers["F"]
employers["%_unknown"] = round(employers["total_gendered"] / (employers["UNKNOWN"] + employers["total_gendered"])* 100, 2)

In [74]:
included_orgs = employers[employers["total_gendered"].gt(300)].index

In [79]:
employer_avg = (data
.loc[data["employer"].isin(included_orgs), :]
.pivot_table(index="employer", values="salary", columns="gender", aggfunc="mean")
)

gender,F,M,UNKNOWN
employer,,,
ALGOMA DISTRICT SCHOOL BOARD,"107,740.80","110,466.52","107,627.53"
ALGONQUIN & LAKESHORE CATHOLIC DISTRICT SCHOOL BOARD,"107,644.35","110,302.79","109,060.36"
ALGONQUIN COLLEGE OF APPLIED ARTS & TECHNOLOGY,"119,323.42","117,602.71","111,351.31"
ATTORNEY GENERAL,"173,546.32","185,128.43","161,273.08"
AVON MAITLAND DISTRICT SCHOOL BOARD,"106,496.67","108,493.00","107,134.05"
...,...,...,...
WINDSOR–ESSEX CATHOLIC DISTRICT SCHOOL BOARD,"106,972.66","108,672.03","106,841.77"
WORKPLACE SAFETY & INSURANCE BOARD,"113,427.52","121,038.77","118,187.98"
YORK CATHOLIC DISTRICT SCHOOL BOARD,"105,773.73","108,995.56","105,728.09"


THis time, let's also add another column to calculate the difference between the average salary for men and for women, then sort by that category.

In [81]:
employer_avg["diff"] = employer_avg["M"] - employer_avg["F"]

employer_avg.sort_values("diff", ascending=False).head()

gender,F,M,UNKNOWN,diff
employer,,,,
THE HOSPITAL FOR SICK CHILDREN,"134,284.00","180,429.00","164,794.00","46,145.00"
SINAI HEALTH SYSTEM,"125,670.00","165,116.00","133,841.00","39,446.00"
GRAND RIVER HOSPITAL CORPORATION,"117,412.00","145,796.00","131,944.00","28,384.00"
HAMILTON HEALTH SCIENCES,"118,083.00","139,945.00","129,009.00","21,862.00"
MCMASTER UNIVERSITY,"155,117.00","176,454.00","169,482.00","21,337.00"
